In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Autoclave batches dataset (2).csv')

C:\Users\91751\AppData\Local\Temp\ipykernel_18524\3187631371.py:1: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Autoclave batches dataset (2).csv')


In [3]:
df['DataLabel'] = 0

# Initialize the first label
current_label = 1

# Now we'll iterate through the dataframe.
# Whenever we encounter 'CYCLE END' in the 'Phase' column,
# we'll increment the 'DataLabel' for subsequent rows.

for index, row in df.iterrows():
    df.at[index, 'DataLabel'] = current_label
    if row['Phase'] == 'CYCLE END':
        current_label += 1


df.drop(columns=['TPR1', 'TPR2','TER2','TER3','TER4','TE7','TER6','TPR2'], inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1042808 entries, 0 to 1042807
Data columns (total 19 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Time            1042808 non-null  object 
 1   TP              1042808 non-null  float64
 2   TE1             977250 non-null   object 
 3   TE2             1042808 non-null  object 
 4   TE3             1042797 non-null  object 
 5   TE4             977239 non-null   object 
 6   TE6             35418 non-null    float64
 7   Program Number  1042808 non-null  int64  
 8   Phase           1042808 non-null  object 
 9   Min_ster_Temp   1042808 non-null  float64
 10  Max_ster_Temp   1042808 non-null  float64
 11  Batch Status    1042808 non-null  object 
 12  Log name        1042808 non-null  object 
 13  PAD             833682 non-null   float64
 14  TER1            949234 non-null   float64
 15  TPR3            2113 non-null     float64
 16  TPR4            2113 non-null     fl

In [5]:
# coerce the error and make it float type TE1
# prompt: convert datatype of TE1 to float and do error coerce

df['TE1'] = pd.to_numeric(df['TE1'], errors='coerce').astype(float)
df['TE2'] = pd.to_numeric(df['TE2'], errors='coerce').astype(float)
df['TE3'] = pd.to_numeric(df['TE3'], errors='coerce').astype(float)
df['TE4'] = pd.to_numeric(df['TE4'], errors='coerce').astype(float)

In [6]:

# Ensure 'Time' column is in datetime format
df['Time'] = pd.to_datetime(df['Time'])

# Extract hour from 'Time'
df['Hour'] = df['Time'].dt.hour


In [7]:
df.tail()

,Time,TP,TE1,TE2,TE3,TE4,TE6,Program Number,Phase,Min_ster_Temp,Max_ster_Temp,Batch Status,Log name,PAD,TER1,TPR3,TPR4,TE17,DataLabel,Hour
1042803,2021-04-05 16:42:00,0.88,52.3,113.3,113.3,113.3,NaN,16,RETURN TO ATMOSPHERIC PRESSURE,121.7,122.3,OK,M3-ACV-0301968.LOG,0.04,52.3,NaN,NaN,NaN,1860,16
1042804,2021-04-05 16:42:00,0.88,51.9,113.3,113.3,113.2,NaN,16,RETURN TO ATMOSPHERIC PRESSURE,121.7,122.3,OK,M3-ACV-0301968.LOG,0.04,51.9,NaN,NaN,NaN,1860,16
1042805,2021-04-05 16:43:00,0.88,51.6,113.3,113.3,113.2,NaN,16,RETURN TO ATMOSPHERIC PRESSURE,121.7,122.3,OK,M3-ACV-0301968.LOG,0.04,51.6,NaN,NaN,NaN,1860,16
1042806,2021-04-05 16:43:00,0.89,51.5,113.3,113.3,113.2,NaN,16,RETURN TO ATMOSPHERIC PRESSURE,121.7,122.3,OK,M3-ACV-0301968.LOG,0.04,51.5,NaN,NaN,NaN,1860,16
1042807,2021-04-05 16:43:00,0.89,51.5,113.3,113.3,113.2,NaN,16,CYCLE END,121.7,122.3,OK,M3-ACV-0301968.LOG,0.04,51.5,NaN,NaN,NaN,1860,16


In [8]:
# make a dataframe with datalabel and Batch status of only CYCLE END phase for each batch, also take Time for that CYCLE END phase
# prompt: create a dataframe with columns 'DataLabel', 'BatchStatus', 'Time' for only 'CYCLE END' phase

df_cycle_end = df[df['Phase'] == 'CYCLE END'][['DataLabel', 'Batch Status', 'Time']]
df_cycle_end.head()


,DataLabel,Batch Status,Time
318,1,FAILED,2019-01-09 09:42:00
667,2,OK,2019-01-09 11:30:00
988,3,OK,2019-01-09 12:32:00
1321,4,OK,2019-01-09 14:00:00
1679,5,OK,2019-01-09 16:10:00


In [9]:
df_cycle_end['Batch Status'].value_counts()

OK        1754
FAILED     106
Name: Batch Status, dtype: int64

In [10]:

# Assign category based on hour
df['Category'] = df.apply(lambda row: 'Night' if 22 <= row['Hour'] or row['Hour'] < 6 else 'Day', axis=1)

# Get the first occurrence of each DataLabel which represents the start of a batch
first_occurrences = df.groupby('DataLabel').nth(0)

# Calculate the number of failed batches that started at night
night_batches = first_occurrences[first_occurrences['Category'] == 'Night']
failed_night_batches = night_batches[night_batches['Batch Status'] == 'FAILED']

In [11]:
print(f'Number of failed batches that started at night: {len(failed_night_batches)}')
print(f'No. of total batches at night: {len(night_batches)}')

Number of failed batches that started at night: 5
No. of total batches at night: 243


In [12]:
df['Batch Status Encoded'] = df['Batch Status'].apply(lambda x: 1 if x == 'OK' else 0)

In [13]:
# Filter rows where Phase is 'STERILIZATION' and calculate the mean temperature for 'STERILIZATION' phase
sterilization_avg_temp = df[df['Phase'] == 'STERILIZATION'].groupby('DataLabel')[['TP','TE1', 'TE2', 'TE3', 'TE4']].mean().reset_index()

# Filter rows where Phase is 'CYCLE END' and select only 'Data Label' and 'Program Number' columns
cycle_end_program = df[df['Phase'] == 'CYCLE END'][['DataLabel', 'Program Number', 'Batch Status','Min_ster_Temp','Max_ster_Temp']]

# Drop duplicate rows in case there are multiple 'CYCLE END' phases for the same 'Data Label'
cycle_end_program = cycle_end_program.drop_duplicates(subset='DataLabel')

# Merge sterilization_avg_temp and cycle_end_program on 'Data Label' to create a new DataFrame
result_df = sterilization_avg_temp.merge(cycle_end_program, on='DataLabel', how='left')

# Identify data labels without 'STERILIZATION' phase
missing_sterilization_labels = df[~df['DataLabel'].isin(result_df['DataLabel'])]['DataLabel'].unique()

# Create a DataFrame for missing data labels with default temperature values and updated 'Batch Status'
missing_df = pd.DataFrame(columns=['DataLabel', 'TP','TE1', 'TE2', 'TE3', 'TE4', 'Program Number', 'Batch Status','Min_ster_Temp','Max_ster_Temp'])

for label in missing_sterilization_labels:
    # Get the batch status and program number for the label
    batch_status = df[df['DataLabel'] == label]['Batch Status'].iloc[0]
    program_number = df[df['DataLabel'] == label]['Program Number'].iloc[0]
    min_temp = df[df['DataLabel'] == label]['Min_ster_Temp'].iloc[0]
    max_temp = df[df['DataLabel'] == label]['Max_ster_Temp'].iloc[0]
    # Set 'Batch Status' to 'Maintenance' or 'Emergency Failure' based on original status
    if batch_status == 'OK':
        status = 'Maintenance'
    elif batch_status == 'FAILED':
        status = 'Emergency Failure'
    # Create a row for missing data label with default temperature values and updated 'Batch Status'
    missing_df = missing_df.append({'DataLabel': label, 'TP': np.nan,'TE1': np.nan, 'TE2': np.nan, 'TE3': np.nan, 'TE4': np.nan, 'Program Number': program_number, 'Batch Status': status,'Min_ster_Temp':min_temp,'Max_ster_Temp':max_temp}, ignore_index=True)

# Concatenate result_df and missing_df to include missing data labels
# also include the time of cycle end phase from df for each data label in result_df


result_df = pd.concat([result_df, missing_df], ignore_index=True)

# Sort the data in result_df based on data labels in ascending order
result_df = result_df.sort_values(by='DataLabel')

overall_avg = df.groupby('DataLabel')[['TE1', 'TE2', 'TE3', 'TE4', 'TP']].mean().reset_index()
overall_avg.columns = ['DataLabel', 'Overall Avg TE1', 'Overall Avg TE2', 'Overall Avg TE3', 'Overall Avg TE4', 'Overall Avg TP']

overall_var = df.groupby('DataLabel')[['TE1', 'TE2', 'TE3', 'TE4', 'TP']].var().reset_index()
overall_var.columns = ['DataLabel', 'Overall Var TE1', 'Overall Var TE2', 'Overall Var TE3', 'Overall Var TE4', 'Overall Var TP']
# Merge overall_avg with result_df on 'DataLabel'
result_df = result_df.merge(overall_avg, on='DataLabel', how='left')
result_df = result_df.merge(overall_var, on='DataLabel', how='left')

# Extract time of cycle end phase for each data label
cycle_end_times = df[df['Phase'] == 'CYCLE END'][['DataLabel', 'Time']]

# Merge cycle end times with result_df on 'DataLabel'
result_df = result_df.merge(cycle_end_times, on='DataLabel', how='left')
# rename cycle_end_times to Time
result_df.rename(columns = {'Time':'Cycle End Time'}, inplace = True)

# Save the result to a new CSV file
result_df.to_csv('results_new.csv', index=False)

# Display the DataFrame with the average sterilization temperature and program number for each 'Data Label'
#print(result_df)
result_df.to_csv('results_new1.csv', index=False)

print("Excel file saved successfully.")

C:\Users\91751\AppData\Local\Temp\ipykernel_18524\823884744.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missing_df = missing_df.append({'DataLabel': label, 'TP': np.nan,'TE1': np.nan, 'TE2': np.nan, 'TE3': np.nan, 'TE4': np.nan, 'Program Number': program_number, 'Batch Status': status,'Min_ster_Temp':min_temp,'Max_ster_Temp':max_temp}, ignore_index=True)
C:\Users\91751\AppData\Local\Temp\ipykernel_18524\823884744.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missing_df = missing_df.append({'DataLabel': label, 'TP': np.nan,'TE1': np.nan, 'TE2': np.nan, 'TE3': np.nan, 'TE4': np.nan, 'Program Number': program_number, 'Batch Status': status,'Min_ster_Temp':min_temp,'Max_ster_Temp':max_temp}, ignore_index=True)
C:\Users\91751\AppData\Local\Temp\ipykernel_18524\823884744.py:31: FutureWarning: The fram

Excel file saved successfully.


In [14]:
result_df.rename(columns = {'Cycle End Time':'Time'}, inplace = True)
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1860 entries, 0 to 1859
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DataLabel        1860 non-null   object        
 1   TP               1590 non-null   float64       
 2   TE1              1590 non-null   float64       
 3   TE2              1590 non-null   float64       
 4   TE3              1590 non-null   float64       
 5   TE4              1590 non-null   float64       
 6   Program Number   1860 non-null   object        
 7   Batch Status     1860 non-null   object        
 8   Min_ster_Temp    1860 non-null   float64       
 9   Max_ster_Temp    1860 non-null   float64       
 10  Overall Avg TE1  1595 non-null   float64       
 11  Overall Avg TE2  1860 non-null   float64       
 12  Overall Avg TE3  1860 non-null   float64       
 13  Overall Avg TE4  1595 non-null   float64       
 14  Overall Avg TP   1860 non-null   float64

In [15]:
# for all rows where 'Batch Status' is 'Maintenance' and Data Label is 2 fill the values of TE1	TE2	TE3	TE4 as 27	27.2	27.4	27.3
# prompt: fill the values of TE1, TE2, TE3, TE4 as 27, 27.2, 27.4, 27.3 for Data Label 2 where Batch Status is 'Maintenance'
result_df.loc[(result_df['Program Number'] == 2) & (result_df['Batch Status'] == 'Maintenance'), ['TP','TE1', 'TE2', 'TE3', 'TE4']] = [3.15, 27, 27.2, 27.4, 27.3]

# for all rows where 'Batch Status' is 'Maintenance' and Program number is 1 fill the values of TP TE1	TE2	TE3	TE4 as 0.08	19.7	19.7	19.8	19.8
# prompt: fill the values of TP, TE1, TE2, TE3, TE4 as 0.08, 19.7, 19.7, 19.8, 19.8 for Program Number 1 where Batch Status is 'Maintenance'
result_df.loc[(result_df['Program Number'] == 1) & (result_df['Batch Status'] == 'Maintenance'), ['TP','TE1', 'TE2', 'TE3', 'TE4']] = [0.08, 19.7, 19.7, 19.8, 19.8]


# for all rows where 'Batch Status' is 'Emergency Failure' fill the values of TP TE1	TE2	TE3	TE4 as 2	96.68571429	118.6714286	118.5	118.6
# prompt: fill the values of TP, TE1, TE2, TE3, TE4 as 2, 96.68571429, 118.6714286, 118.5, 118.6 where Batch Status is 'Emergency Failure'
result_df.loc[result_df['Batch Status'] == 'Emergency Failure', ['TP','TE1', 'TE2', 'TE3', 'TE4']] = [2, 96.68571429, 118.6714286, 118.5, 118.6]

In [16]:
result_df.head()

,DataLabel,TP,TE1,TE2,TE3,TE4,Program Number,Batch Status,Min_ster_Temp,Max_ster_Temp,...,Overall Avg TE2,Overall Avg TE3,Overall Avg TE4,Overall Avg TP,Overall Var TE1,Overall Var TE2,Overall Var TE3,Overall Var TE4,Overall Var TP,Time
0,1,2.110000,0.000000,122.100000,122.200000,122.100000,8,FAILED,122.1,122.2,...,114.396238,115.152978,114.968966,1.547931,2803.869759,131.966338,112.167656,115.738436,0.657171,2019-01-09 09:42:00
1,2,2.109180,121.744262,121.963115,122.161475,122.013934,8,OK,121.2,122.2,...,112.804585,113.555301,113.648711,1.100487,874.492528,127.465898,114.342824,109.503023,0.728475,2019-01-09 11:30:00
2,3,2.108934,121.779508,122.040164,122.269672,122.140164,3,OK,121.2,122.4,...,113.893458,114.797819,114.575701,1.074486,925.593441,86.884676,77.099151,81.663595,0.849215,2019-01-09 12:32:00
3,4,2.109024,121.760163,121.968293,122.203252,122.082927,11,OK,121.2,122.3,...,113.450450,114.356156,114.064565,1.073874,967.878284,89.466001,81.164036,86.763680,0.824727,2019-01-09 14:00:00
4,5,2.109754,121.595902,121.831148,122.027869,121.883607,10,OK,121.2,122.1,...,102.759497,102.860056,103.077933,1.959721,879.638523,473.956702,492.689689,450.135114,0.143143,2019-01-09 16:10:00


In [17]:
# For overall temperature TE1 if there are any missing values fill them with the mean of overall temperature of TE2, TE3
# and TE4
# prompt: fill missing values of TE1 with the mean of overall temperature of TE2, TE3, and TE4
result_df['Overall Avg TE1'] = result_df['Overall Avg TE1'].fillna(result_df[['Overall Avg TE2', 'Overall Avg TE3', 'Overall Avg TE4']].mean(axis=1))
# do same for TE4 impute average of overall TE2 and overall TE3
result_df['Overall Avg TE4'] = result_df['Overall Avg TE4'].fillna(result_df[['Overall Avg TE2', 'Overall Avg TE3']].mean(axis=1))

# For overall temperature TE1 if there are any missing values fill them with the variance of overall temperature of TE2, TE3 and TE4
result_df['Overall Var TE1'] = result_df['Overall Var TE1'].fillna(result_df[['Overall Var TE2', 'Overall Var TE3']].mean(axis=1))
result_df['Overall Var TE4'] = result_df['Overall Var TE4'].fillna(result_df[['Overall Var TE2', 'Overall Var TE3']].mean(axis=1))


In [18]:
result_df.head()

,DataLabel,TP,TE1,TE2,TE3,TE4,Program Number,Batch Status,Min_ster_Temp,Max_ster_Temp,...,Overall Avg TE2,Overall Avg TE3,Overall Avg TE4,Overall Avg TP,Overall Var TE1,Overall Var TE2,Overall Var TE3,Overall Var TE4,Overall Var TP,Time
0,1,2.110000,0.000000,122.100000,122.200000,122.100000,8,FAILED,122.1,122.2,...,114.396238,115.152978,114.968966,1.547931,2803.869759,131.966338,112.167656,115.738436,0.657171,2019-01-09 09:42:00
1,2,2.109180,121.744262,121.963115,122.161475,122.013934,8,OK,121.2,122.2,...,112.804585,113.555301,113.648711,1.100487,874.492528,127.465898,114.342824,109.503023,0.728475,2019-01-09 11:30:00
2,3,2.108934,121.779508,122.040164,122.269672,122.140164,3,OK,121.2,122.4,...,113.893458,114.797819,114.575701,1.074486,925.593441,86.884676,77.099151,81.663595,0.849215,2019-01-09 12:32:00
3,4,2.109024,121.760163,121.968293,122.203252,122.082927,11,OK,121.2,122.3,...,113.450450,114.356156,114.064565,1.073874,967.878284,89.466001,81.164036,86.763680,0.824727,2019-01-09 14:00:00
4,5,2.109754,121.595902,121.831148,122.027869,121.883607,10,OK,121.2,122.1,...,102.759497,102.860056,103.077933,1.959721,879.638523,473.956702,492.689689,450.135114,0.143143,2019-01-09 16:10:00


In [23]:
result_df.isnull().sum()
# check null values in result df



# fill the null values in 'overall var TE1' with 'overall avg TE1'
result_df['Overall Var TE1'] = result_df['Overall Var TE1'].fillna(result_df['Overall Avg TE1'])

# fill the null values in 'overall var TE4' with 'overall avg TE4'
result_df['Overall Var TE4'] = result_df['Overall Var TE4'].fillna(result_df['Overall Avg TE4'])

# fill the null values in 'overall var TE2' with 'overall avg TE2'
result_df['Overall Var TE2'] = result_df['Overall Var TE2'].fillna(result_df['Overall Avg TE2'])

# fill the null values in 'overall var TE3' with 'overall avg TE3'
result_df['Overall Var TE3'] = result_df['Overall Var TE3'].fillna(result_df['Overall Avg TE3'])

# fill the null values in 'overall var TP' with 'overall avg TP'
result_df['Overall Var TP'] = result_df['Overall Var TP'].fillna(result_df['Overall Avg TP'])

result_df.isnull().sum()

DataLabel          0
TP                 0
TE1                0
TE2                0
TE3                0
TE4                0
Program Number     0
Batch Status       0
Min_ster_Temp      0
Max_ster_Temp      0
Overall Avg TE1    0
Overall Avg TE2    0
Overall Avg TE3    0
Overall Avg TE4    0
Overall Avg TP     0
Overall Var TE1    0
Overall Var TE2    0
Overall Var TE3    0
Overall Var TE4    0
Overall Var TP     0
Time               0
dtype: int64

In [24]:
result_df.info()
# get an excel sheet with the updated values
result_df.to_csv('results_new2.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1860 entries, 0 to 1859
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DataLabel        1860 non-null   object        
 1   TP               1860 non-null   float64       
 2   TE1              1860 non-null   float64       
 3   TE2              1860 non-null   float64       
 4   TE3              1860 non-null   float64       
 5   TE4              1860 non-null   float64       
 6   Program Number   1860 non-null   object        
 7   Batch Status     1860 non-null   object        
 8   Min_ster_Temp    1860 non-null   float64       
 9   Max_ster_Temp    1860 non-null   float64       
 10  Overall Avg TE1  1860 non-null   float64       
 11  Overall Avg TE2  1860 non-null   float64       
 12  Overall Avg TE3  1860 non-null   float64       
 13  Overall Avg TE4  1860 non-null   float64       
 14  Overall Avg TP   1860 non-null   float64

In [13]:
# get all Phases for data label 30
print(df[df['DataLabel'] == 30]['Phase'].unique())

df[df['DataLabel'] == 30]['Batch Status'].unique()

['PREPARE AUTOCLAVE' 'DEPRESSURIZE BY' 'CHAMBER VACUUM' 'RETURN TO'
 'CYCLE END']


array(['OK'], dtype=object)

In [14]:
# get all Phases for data label 12
print(df[df['DataLabel'] == 12]['Phase'].unique())

# get batch status for data label 12
df[df['DataLabel'] == 12]['Batch Status'].unique()


['PREPARE AUTOCLAVE' 'DEPRESSURIZE BY VACUUM PUMP'
 'TIMED VACUUM, STEAM INJECTION' 'EMERGENCY' 'CYCLE END']


array(['FAILED'], dtype=object)

In [15]:
# get number of unique datalabel in which Phase EMERGENCY is PRESENT and get corresponding OK and FAILED batch status for that datalabel using CYCLE END Phase
emergency_batches = df[df['Phase'] == 'EMERGENCY']['DataLabel'].unique()
print(f"Data Labels with EMERGENCY phase: {emergency_batches}")
# for these datalabel get the corresponding batch status using CYCLE END phase
emergency_batches_status = df[df['DataLabel'].isin(emergency_batches) & (df['Phase'] == 'CYCLE END')][['DataLabel', 'Batch Status']]
print(emergency_batches_status)

Data Labels with EMERGENCY phase: [   1   12   14   26   32   35   52   56  132  143  673 1457 1798]
         DataLabel Batch Status
318              1       FAILED
4360            12       FAILED
5150            14           OK
9070            26       FAILED
12246           32       FAILED
14210           35       FAILED
19648           52       FAILED
20642           56       FAILED
57604          132       FAILED
61873          143           OK
329984         673       FAILED
796051        1457           OK
1007968       1798           OK
